In [1]:
using DataFrames, CSV, DelimitedFiles, Plots, FFTW
include("/home/kirk/Documents/research/CHANDRATest/DJM.jl")

Main.DJM

In [2]:
pows12=CSV.read("cxc.fftinfo.all12")

,obsid,srcnum,startdate,exptime,ra,dec,freq,power
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1026,26,51833.8,6458.48,19.9382,-73.4501,0.00123975,13.2664
2,1028,105,51856.5,6324.74,19.038,-73.414,0.000316292,13.824
3,1028,113,51856.5,6324.74,18.9547,-73.4035,0.00110873,31.303
4,1028,113,51856.5,6324.74,18.9547,-73.4035,0.00126712,12.1519
5,1028,38,51856.5,6324.74,19.8812,-73.5169,0.00110816,21.5887
6,1028,88,51856.5,6324.74,19.1853,-73.4323,0.000316454,12.6433
7,1029,13,52024.6,6120.26,19.6143,-73.4658,0.00115807,56.0898
8,1030,14,52025.6,6152.72,19.6181,-73.4675,0.00114639,74.6027
9,1030,14,52025.6,6152.72,19.6181,-73.4675,0.00212901,18.2923


In [3]:
using Formatting
function genProbMatrix(powList)
    df=0.0001
    nk=ceil(maximum(powList.freq[:])/df)
    justPow=powList.power[:]    
    probMatrix=zeros(Int(nk),Int(ceil(maximum(justPow)-minimum(justPow))+1)) #freq num x pow range, +1 because min-min=0
    counter=0
    for i=1:length(justPow) #THIS DOES NOT ACCOUNT FOR PAIRS IN SAME PLACE
        print(format(i/length(justPow)*100,precision=2),"% complete part 1\r") #output % tracker
        freqInd=Int(ceil(powList.freq[i]/df))
        powInd=Int(ceil(powList.power[i]-minimum(powList.power[:])))+1 #+1 because index starts at 1
        probMatrix[freqInd,powInd]+=1
        counter+=1
    end
    
    #this accounts for pairs
    freqIndTrack=[] #set up to keep track of indices where we've already been
    powIndTrack=[]
    for i=1:length(justPow)
        print("\r")
        print(format(i/length(justPow)*100,precision=2),"% complete part 2\r") #output % tracker
        freq1Ind=Int(ceil(powList.freq[i]/df))
        pow1Ind=Int(ceil(powList.power[i]-minimum(powList.power[:])))+1 #+1 because min-min=0
        for j=i:(length(justPow)-1)#-1 because want i==j but still want pair not to be same src
            rdist=DJM.gcdist(powList.ra[j+1],powList.dec[j+1],powList.ra[i],powList.dec[i])
            if (60.0*rdist)<0.01666 #in the same place
                freq2Ind=Int(ceil(powList.freq[j+1]/df))
                pow2Ind=Int(ceil(powList.power[j+1]-minimum(powList.power[:])))+1 #+1 because min-min =0
                if j==i #so we only count i once
                    if freq1Ind in freqIndTrack && pow1Ind in powIndTrack && freq2Ind in freqIndTrack && pow2Ind in powIndTrack
                        continue
                    else
                        probMatrix[freq1Ind,pow1Ind]-=1 #take away one because it's a loc pair
                        probMatrix[freq2Ind,pow2Ind]-=1
                        counter-=2 #take 2 off counter
                        push!(freqIndTrack,freq1Ind) #add indices to our list 
                        push!(freqIndTrack,freq2Ind) #so that we don't oversubtract
                        push!(powIndTrack,pow1Ind)
                        push!(powIndTrack,pow2Ind)
                    end

                else
                    if freq2Ind in freqIndTrack && pow2Ind in powIndTrack
                        continue
                    else
                        probMatrix[freq2Ind,pow2Ind]-=1 
                        counter-=1
                        push!(freqIndTrack,freq2Ind)
                        push!(powIndTrack,pow2Ind)
                    end
                end
            end
        end
    end
    #this makes probabilities include greater than                            
    for i=1:length(probMatrix[:,1]) #num rows
        print("\r")
        print(format(i/length(probMatrix[:,1])*100,precision=2),"% complete part 3\r") #output % tracker
        currentRow=probMatrix[i,:]
        for j=1:length(currentRow) 
            probMatrix[i,j]+=sum(currentRow[(j+1):end]) #so each cell holds num >= to pow
        end
    end
    probMatrix=probMatrix./counter #rescales by dividing out total number of sources
    return probMatrix
end

genProbMatrix (generic function with 1 method)

In [4]:
test=genProbMatrix(pows12)

1544×1886 Array{Float64,2}:
 0.288103     0.288103     0.27717      …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.10418      0.10418      0.096463        0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0623794    0.0623794    0.0572347       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0115756    0.0115756    0.0096463       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00321543      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.000643087  0.000643087  0.000643087  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00321543   0.00321543   0.00257235      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00128617   0.00128617   0.00128617      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00257235      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00385852      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0604502    0.0604502    0.0508039    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0102894    0.0102894    0.0102894       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.000643087  0.000643087  0.000643087     0.0  

In [5]:
test=genProbMatrix(pows12)

1544×1886 Array{Float64,2}:
 0.288103     0.288103     0.27717      …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.10418      0.10418      0.096463        0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0623794    0.0623794    0.0572347       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0115756    0.0115756    0.0096463       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00321543      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.000643087  0.000643087  0.000643087  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00321543   0.00321543   0.00257235      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00128617   0.00128617   0.00128617      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00257235      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00385852      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0604502    0.0604502    0.0508039    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0102894    0.0102894    0.0102894       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.000643087  0.000643087  0.000643087     0.0  

In [6]:
manip=test

1544×1886 Array{Float64,2}:
 0.288103     0.288103     0.27717      …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.10418      0.10418      0.096463        0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0623794    0.0623794    0.0572347       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0115756    0.0115756    0.0096463       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00321543      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.000643087  0.000643087  0.000643087  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00321543   0.00321543   0.00257235      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00128617   0.00128617   0.00128617      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00257235      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00385852   0.00385852   0.00385852      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0604502    0.0604502    0.0508039    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0102894    0.0102894    0.0102894       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.000643087  0.000643087  0.000643087     0.0  

In [7]:
manip=replace!(manip,0=>1e3)

1544×1886 Array{Float64,2}:
    0.288103        0.288103     …  1000.0  1000.0  1000.0  1000.0
    0.10418         0.10418         1000.0  1000.0  1000.0  1000.0
    0.0623794       0.0623794       1000.0  1000.0  1000.0  1000.0
    0.0115756       0.0115756       1000.0  1000.0  1000.0  1000.0
    0.00385852      0.00385852      1000.0  1000.0  1000.0  1000.0
    0.000643087     0.000643087  …  1000.0  1000.0  1000.0  1000.0
    0.00321543      0.00321543      1000.0  1000.0  1000.0  1000.0
    0.00128617      0.00128617      1000.0  1000.0  1000.0  1000.0
    0.00385852      0.00385852      1000.0  1000.0  1000.0  1000.0
    0.00385852      0.00385852      1000.0  1000.0  1000.0  1000.0
    0.0604502       0.0604502    …  1000.0  1000.0  1000.0  1000.0
    0.0102894       0.0102894       1000.0  1000.0  1000.0  1000.0
    0.000643087     0.000643087     1000.0  1000.0  1000.0  1000.0
    ⋮                            ⋱                             ⋮  
 1000.0          1000.0           

In [8]:
x=minimum(manip)

0.0006430868167202572

In [9]:
numTrials=250*1544 #numpairs x numfreq

386000

In [10]:
pows8=CSV.read("cxc.fftinfo.all8")

,obsid,srcnum,startdate,exptime,ra,dec,freq,power
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1026,26,51833.8,6458.48,19.9382,-73.4501,0.00123975,13.2664
2,1026,8,51833.8,6458.48,19.5522,-73.6193,0.154427,9.16655
3,1027,107,51855.0,6133.27,18.8667,-73.3961,0.0185454,9.14431
4,1027,107,51855.0,6133.27,18.8667,-73.3961,0.144096,8.02378
5,1027,36,51855.0,6133.27,19.782,-73.5006,0.0813522,8.00401
6,1027,40,51855.0,6133.27,19.7033,-73.4917,0.0417401,8.04439
7,1027,54,51855.0,6133.27,19.5056,-73.4696,0.14484,8.14304
8,1027,58,51855.0,6133.27,19.4489,-73.4632,0.0598763,8.05626
9,1027,91,51855.0,6133.27,19.0827,-73.4213,0.0622972,8.75528


In [11]:
prob8=genProbMatrix(pows8)

98.22% complete part 210.37% complete part 20.87% complete part 21.03% complete part 21.19% complete part 21.35% complete part 21.85% complete part 22.17% complete part 22.33% complete part 22.49% complete part 22.65% complete part 22.81% complete part 23.15% complete part 23.64% complete part 24.14% complete part 24.30% complete part 24.64% complete part 24.98% complete part 2% complete part 25.95% complete part 26.13% complete part 26.29% complete part 26.45% complete part 26.61% complete part 27.09% complete part 27.25% complete part 27.41% complete part 27.57% complete part 27.73% complete part 28.07% complete part 28.23% complete part 28.57% complete part 28.90% complete part 29.06% complete part 29.22% complete part 210.20% complete part 210.32% complete part 210.47% complete part 210.70% complete part 212.17% complete part 212.85% complete part 213.01% complete part 213.36% complete part 213.90% complete part 214.46% complete part 214.61% complete part 214.78% complete part 214.

Excessive output truncated after 524291 bytes.

99.09

1546×1890 Array{Float64,2}:
 0.0634798    0.0634798    0.058843     …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0258335    0.0258335    0.0231839       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0166703    0.0166703    0.0140208       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00872157   0.00872157   0.00596158      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00287039   0.00287039   0.00198719      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00209759   0.00209759   0.0012144    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00231839   0.00231839   0.0014352       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00242879   0.00242879   0.001104        0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0014352    0.0014352    0.001104        0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00264959   0.00264959   0.00187679      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0186575    0.0186575    0.0155664    …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.00419519   0.00419519   0.00309119      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.000993597  0.000993597  0.000441599     0.0  

In [12]:
pow1=ceil(pows8.power[381])

39.0

In [13]:
manip8=prob8
manip8=replace!(manip8,0=>1e3)
min8=minimum(manip8)
numTrials8=1546*250 #numfreq x numpairs

386500

In [14]:
pow2=ceil(pows8.power[1283])

16.0

In [15]:
df=0.0001
pow1Ind=Int(ceil(pow1-minimum(pows8.power[:]))+1)
pow2Ind=Int(ceil(pow2-minimum(pows8.power[:]))+1)
freq1Ind=Int(ceil(pows8.freq[381]/df))
freq2Ind=Int(ceil(pows8.freq[1283]/df))
expected=numTrials8*prob8[freq1Ind,pow1Ind]*prob8[freq2Ind,pow2Ind]

0.02355346845756068

In [16]:
pow2T=pows8.startdate[1283]
pow1T=pows8.startdate[381]
delta=abs(pow2T-pow1T) #days

0.8212941491874517

In [18]:
freq2Ind

33

In [22]:
prob8[1546,1]

0.00011039964672113049

1546×1890 Array{Float64,2}:
    0.0634798       0.0634798    …  1000.0  1000.0  1000.0  1000.0
    0.0258335       0.0258335       1000.0  1000.0  1000.0  1000.0
    0.0166703       0.0166703       1000.0  1000.0  1000.0  1000.0
    0.00872157      0.00872157      1000.0  1000.0  1000.0  1000.0
    0.00287039      0.00287039      1000.0  1000.0  1000.0  1000.0
    0.00209759      0.00209759   …  1000.0  1000.0  1000.0  1000.0
    0.00231839      0.00231839      1000.0  1000.0  1000.0  1000.0
    0.00242879      0.00242879      1000.0  1000.0  1000.0  1000.0
    0.0014352       0.0014352       1000.0  1000.0  1000.0  1000.0
    0.00264959      0.00264959      1000.0  1000.0  1000.0  1000.0
    0.0186575       0.0186575    …  1000.0  1000.0  1000.0  1000.0
    0.00419519      0.00419519      1000.0  1000.0  1000.0  1000.0
    0.000993597     0.000993597     1000.0  1000.0  1000.0  1000.0
    ⋮                            ⋱                                
    0.000331199     0.000331199   